In [168]:
import pandas as pd
import numpy as np
import locale
import csv

In [169]:
#IMPORTANDO BASE DE DADOS, criando um lista bdagro e removendo colunas indesejadas
lista_bd_agro=['CHAVE',	'CLIENTE',	'SAFRA',	'OBJETIVO',	'TP_PROP',	'FAZENDA',	'SETOR',	'SECAO',	'BLOCO',	'PIVO',	'DESC_FAZ',	'TALHAO',	'VARIEDADE',	'MATURACAO',	'AMBIENTE',	'IRRIGACAO',	'ESTAGIO',	'GRUPO_DASH',	'GRUPO_NDVI',	'NMRO_CORTE',	'DESC_CANA',	'AREA_BD',	'A_EST_MOAGEM',	'A_COLHIDA',	'A_EST_MUDA',	'A_MUDA',	'TCH_EST',	'TC_EST',	'TCH_REST',	'TC_REST',	'TCH_REAL',	'TC_REAL',	'DT_CORTE',	'DT_ULT_CORTE',	'DT_PLANTIO',	'IDADE_CORTE',	'ATR',	'ATR_EST','TAH']
banco_aralco = pd.read_csv('input/base_talhao_202507231055.csv', sep=',', encoding='utf-8')
estagios = pd.read_excel('C:/Users/luan.faria/Desktop/cod_luan/cod/SIGMA/cod/bd_agro_automatico/estagios.xlsx')
bd_aralco = banco_aralco
bd_agro = pd.DataFrame(columns=lista_bd_agro)


In [170]:
#concatenar ID
bd_aralco['chave'] = bd_aralco['cod_fazenda'].astype(str) + '_' + bd_aralco['cod_talhao'].astype(str)


In [171]:
bd_aralco['numero_corte'] = bd_aralco['numero_corte'].apply(
    lambda x: int(x) if str(x).isdigit() else x)


In [172]:
#renomear tp_prop
bd_aralco['tipo_fazenda'] = bd_aralco['tipo_fazenda'].replace({
    'PROPRIA': 'PRÓPRIAS',
    'FORNECEDORES': 'FORNECEDORES'})

bd_aralco = bd_aralco.loc[bd_aralco['tipo_fazenda'] != 'SPOT']

bd_aralco['amb_producao'] = bd_aralco['amb_producao'].fillna('ADEF')
bd_aralco['manejo'] = bd_aralco['manejo'].fillna('ADEF')
bd_aralco['manejo'] = bd_aralco['manejo'].replace({'A DEFINIR':'ADEF', 'MEDIA':'MÉDIA'})


C:\Users\luan.faria\AppData\Local\Temp\ipykernel_17176\1607425176.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bd_aralco['amb_producao'] = bd_aralco['amb_producao'].fillna('ADEF')
C:\Users\luan.faria\AppData\Local\Temp\ipykernel_17176\1607425176.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bd_aralco['manejo'] = bd_aralco['manejo'].fillna('ADEF')
C:\Users\luan.faria\AppData\Local\Temp\ipykernel_17176\1607425176.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [173]:
bd_aralco = bd_aralco[
    ~((bd_aralco['areamuda'] == 0) & 
      (bd_aralco['areaproducao'] == 0) & 
      (bd_aralco['area_reforma'] == 0))
]


In [174]:
#RESOLVENDO O PROBLEMAS DAQUELES DUPLICADOS 
total_area = bd_aralco['areaproducao'].sum()
print(f'Área total de produção: {total_area:.2f} ha')

# 1. Remove as duplicatas com areaproducao = 0
filtro = (bd_aralco.duplicated(subset='chave', keep=False)) & (bd_aralco['areaproducao'] == 0) & (bd_aralco['areamuda'] == 0)
bd_aralco = bd_aralco[~filtro]

### 2. Remove duplicatas mantendo a primeira (caso ainda tenha alguma)
bd_aralco = bd_aralco.drop_duplicates(subset='chave', keep='first')

#verificar se tem duplicados 
total_area = bd_aralco['areaproducao'].sum()
print(f'Área total de produção: {total_area:.2f} ha')
numero_duplicados = bd_aralco['chave'].duplicated().sum()
print(f'Número de duplicatas na coluna "id": {numero_duplicados}')

Área total de produção: 61707.41 ha
Área total de produção: 61702.41 ha
Número de duplicatas na coluna "id": 0


In [175]:
#colhidos total
# Definindo as condições de objetivo
condicoes = [
    (bd_aralco['encerrado'] == 'S')]

# Definindo os valores correspondentes para cada condição
valores = [bd_aralco['area_realizada']]

# Aplicando as condições e atribuindo o valor na nova coluna 'OBJETIVO'
bd_aralco['area_realizada'] = np.select(condicoes, valores, default=0)

#valor
total_area_real = bd_aralco['area_realizada'].sum()
print(f'Área total de produção: {total_area_real:.2f} ha')


Área total de produção: 23949.06 ha


In [176]:
# Definindo as condições de objetivo
condicoes = [
    (bd_aralco['areamuda'] > 0) & (bd_aralco['areaproducao'] > 0),
    (bd_aralco['areamuda'] > 0) & (bd_aralco['areaproducao'] == 0),
    (bd_aralco['areamuda'] == 0) & (bd_aralco['areaproducao'] > 0)]
                                   

# Definindo os valores correspondentes para cada condição
valores = ['MOAGEM/MUDA','MUDA','MOAGEM']

# Aplicando as condições e atribuindo o valor na nova coluna 'OBJETIVO'
bd_aralco['OBJETIVO'] = np.select(condicoes, valores, default='REFORMA')
bd_aralco['area_util'] = bd_aralco['areaplantada'] - bd_aralco['areadiversa']

In [177]:
bd_aralco = bd_aralco[bd_aralco['OBJETIVO'] != 'ADEF'].reset_index(drop=True)
#esses adef são areas de dano
area_por_objetivo = bd_aralco.groupby('OBJETIVO')['area_util'].sum()
print(area_por_objetivo)

OBJETIVO
MOAGEM         59302.17
MOAGEM/MUDA     3727.73
MUDA            1463.64
REFORMA        13128.02
Name: area_util, dtype: float64


In [178]:
# Ajuste o nome da coluna, se necessário
coluna_estagios = 'ESTAGIO'  # Substitua pelo nome correto se for diferente

# Realizando o merge
bd_aralco = pd.merge(
    bd_aralco,
    estagios,
    left_on='numero_corte',  # Coluna no bd_aralco
    right_on=coluna_estagios,  # Coluna no estagios
    how='left'
)

# Removendo a coluna duplicada se necessário
if coluna_estagios in bd_aralco.columns:
    bd_aralco.drop(columns=[coluna_estagios], inplace=True)

In [179]:
bd_agro['CHAVE'] = bd_aralco['chave']
bd_agro['CLIENTE'] = 'ARALCO'
bd_agro['SAFRA'] = bd_aralco['cod_safra']
bd_agro['OBJETIVO'] = bd_aralco['OBJETIVO']
bd_agro['TP_PROP'] = bd_aralco['tipo_fazenda']
bd_agro['FAZENDA'] = bd_aralco['cod_fazenda']
bd_agro['SETOR'] = ''
bd_agro['SECAO'] = ''
bd_agro['BLOCO'] = ''
bd_agro['PIVO'] = ''
bd_agro['DESC_FAZ'] = bd_aralco['fazenda']
bd_agro['TALHAO'] = bd_aralco['cod_talhao']
bd_agro['VARIEDADE'] = bd_aralco['variedade'] 
bd_agro['MATURACAO'] = bd_aralco['manejo']
bd_agro['AMBIENTE'] = bd_aralco['amb_producao']
bd_agro['IRRIGACAO'] = 'ADEF'
bd_agro['TESTE']=bd_aralco['numero_corte']
bd_agro['ESTAGIO'] = bd_aralco['ESTAGIO_25']
bd_agro['GRUPO_DASH'] = bd_aralco['GRUPO_DASH'] 
bd_agro['GRUPO_NDVI'] = bd_aralco['GRUPO_NDVI']
bd_agro['NMRO_CORTE'] = bd_aralco['NMRO_CORTE']
bd_agro['DESC_CANA'] = bd_aralco['DESC_CANA']
bd_agro['AREA_BD'] = bd_aralco['area_util']
bd_agro['A_EST_MOAGEM'] = bd_aralco['areaproducao']
bd_agro['A_COLHIDA'] = bd_aralco['area_realizada']
bd_agro['A_EST_MUDA'] = bd_aralco['areamuda']
bd_agro['A_MUDA'] = bd_aralco['areamuda']

In [180]:
#bd_estiva = bd_estiva.replace('.',',')
bd_agro['TCH_EST'] = bd_aralco['tch_estimado']
bd_agro['TC_EST'] =bd_aralco['areaproducao'] * bd_aralco['tch_estimado']
bd_agro['TCH_REST'] = ''
bd_agro['TC_REST'] = ''
bd_aralco['TC_REAL'] = np.select([bd_aralco['encerrado'] == 'S'], [bd_aralco['producao_realizada']], default=0)
bd_agro['TCH_REAL'] = np.select([bd_aralco['TC_REAL']>0], [bd_aralco['TC_REAL'] / bd_agro['A_COLHIDA']], default=0)
bd_agro['TC_REAL'] = bd_aralco['TC_REAL']


In [185]:
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')  # Para exibir a data em português
bd_agro['DT_CORTE'] =  np.select([bd_aralco['encerrado'] == 'S'], [bd_aralco['dataencerramento']], default=0)
bd_agro['DT_ULT_CORTE'] = bd_aralco['data_corte_safra_anterior']
bd_agro['DT_PLANTIO'] = bd_aralco['dataplantio']
bd_agro['DT_ULT_CORTE'] = np.select([bd_agro['NMRO_CORTE'] != 1], [bd_agro['DT_ULT_CORTE']], default=pd.to_datetime('1900-01-01').date())
bd_agro['DT_CORTE'] = bd_agro['DT_CORTE'].replace(0,pd.to_datetime('1900-01-01').date())
bd_agro['DT_ULT_CORTE'] = bd_agro['DT_ULT_CORTE'].replace(0,pd.to_datetime('1900-01-01').date())
bd_agro['DT_PLANTIO'] = bd_agro['DT_PLANTIO'].replace(0,pd.to_datetime('1900-01-01').date())
bd_agro['DT_ULT_CORTE'] = bd_agro['DT_ULT_CORTE'].fillna(pd.to_datetime('1900-01-01').date())
bd_agro['DT_PLANTIO'] = bd_agro['DT_PLANTIO'].fillna(pd.to_datetime('1900-01-01').date())
bd_agro['DT_CORTE'] = pd.to_datetime(bd_agro['DT_CORTE'])
bd_agro['DT_PLANTIO'] = pd.to_datetime(bd_agro['DT_PLANTIO'])
bd_agro['DT_ULT_CORTE'] = pd.to_datetime(bd_agro['DT_ULT_CORTE'])

#print(bd_agro[['DT_CORTE', 'DT_ULT_CORTE', 'DT_PLANTIO']].head())

In [186]:
bd_agro['IDADE_CORTE'] = np.where(
    bd_agro['NMRO_CORTE'] == 1,
    (bd_agro['DT_CORTE'] - bd_agro['DT_PLANTIO']).dt.days / 30,
    (bd_agro['DT_CORTE'] - bd_agro['DT_ULT_CORTE']).dt.days / 30)
bd_agro.loc[(bd_agro['IDADE_CORTE'] > 35) | (bd_agro['IDADE_CORTE'] < 0), 'IDADE_CORTE'] = 0  

In [183]:
bd_agro['ATR'] = np.select([bd_aralco['encerrado'] == 'S'], [bd_aralco['atr']], default=0)

In [184]:
bd_agro.to_excel('output/BD_AGRO_ARALCO.xlsx', index=False)